# VQA BGE-small search

In [1]:
import pandas as pd
import numpy as np
from transformers import AutoModel, AutoTokenizer
import faiss
import torch
from tqdm.auto import trange
import ast
import pickle
import re
import torch.functional as F

In [2]:
def prepare_text(s):
    """Функция предобработки текста"""
    return re.sub(" +", " ",s.lower().replace('_',' ').strip())

def parse_request(row):
    return row['question']

def make_df(path,is_test=False):
    """"Предобработка данных"""
    data = pd.read_csv(path)
    df = pd.DataFrame()
    
    df['question'] = data['question']
    df['choices'] = data['choices'].apply(ast.literal_eval)
    
    return df

In [3]:
# Загружаем данные
train_data = make_df('all_datasets.csv')

# Сопоставляем запросы
train_data['request'] = train_data.apply(parse_request,axis=1)
train_choices = [prepare_text(x) for x in np.concatenate(train_data['choices'])]

/tmp/ipykernel_988/1048887473.py:10: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path)


In [4]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    
def cls_pooling(model_output):
    return model_output[0][:, 0]

@torch.no_grad()
def create_embeds(texts,model,tokenizer,batch_size=64,max_length=1024):
    """Функция, для получения векторных представлений из текста"""
    features = []
    texts = [prepare_text(x) for x in texts]
    for i in trange(len(texts) // batch_size + 1):
        encs = tokenizer(
            texts[i*batch_size:(i+1)*batch_size],
            truncation=True,
            padding='longest',
            max_length=max_length,
            return_tensors='pt'
        )
        encs['input_ids'] = encs['input_ids'].cuda()
        encs['attention_mask'] = encs['attention_mask'].cuda()
        if 'token_type_ids' in encs.keys():
            encs['token_type_ids'] = encs['token_type_ids'].cuda()
        model_output = model(**encs)
        embeds = cls_pooling(model_output)#mean_pooling(model_output, encs['attention_mask'])
        embeds = torch.nn.functional.normalize(embeds, p=2, dim=1).detach().cpu().numpy()
        features.append(embeds)
    return np.concatenate(features,dtype=np.float32)

In [5]:
encoder = AutoModel.from_pretrained('BAAI/bge-small-en-v1.5').cuda()
tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-small-en-v1.5')

2024-11-10 12:41:23.355954: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-10 12:41:23.356015: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-10 12:41:23.357125: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-10 12:41:23.363088: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-10 12:41:24.307514: W tensorflow/compiler/tf2

In [6]:
graph_state = pickle.load(open('graph.pkl','rb'))

In [7]:
# делаем препроцессинг вершин
concepts = [prepare_text(s)  if type(s) == str else ' ' for s in graph_state['concepts_maper']]

In [8]:
features = create_embeds(concepts,encoder,tokenizer,batch_size=512)

  0%|          | 0/1562 [00:00<?, ?it/s]

In [9]:
# Создаём faiss index для быстрого поиска кандидатов
index = faiss.IndexFlatIP(features.shape[1])
faiss.normalize_L2(features)
index.add(features)
res = faiss.StandardGpuResources()
index = faiss.index_cpu_to_gpu(res, 0, index)
index.ntotal

799272

In [10]:
# Векторизуем запросы
question_features = create_embeds(train_data['request'].tolist(),encoder,tokenizer,batch_size=128,max_length=512)
choice_features = create_embeds(train_choices,encoder,tokenizer,batch_size=128,max_length=512)

  0%|          | 0/3981 [00:00<?, ?it/s]

  0%|          | 0/15921 [00:00<?, ?it/s]

In [11]:
# Ищем по запросам релевантные вершины 
faiss.normalize_L2(question_features)
faiss.normalize_L2(choice_features)
D_q, I_q = index.search(question_features,k=7)
D_ch, I_ch = index.search(choice_features,k=1)

In [12]:
I_ch[:,0]

array([373284, 428188,  49167, ...,    929,   3407,  10932])

In [13]:
state_bge = {
    'question_encodes': I_q,
    'choices_encodes': I_ch[:,0]
}

with open('./vqa_all_state_bge','wb') as f:
    pickle.dump(state_bge,f)

Если у вас мало RAM, то рекомендуется перезагрузить среду

# Lemmas Search

In [2]:
import numpy as np
import pandas as pd
import re
import multiprocessing as mp
from tqdm.auto import tqdm
from tqdm.contrib import tzip
import spacy
import pickle
import ast
from string import punctuation
from spacy.lang.en import stop_words

In [3]:
def prepare_text(s):
    """Функция предобработки текста"""
    return re.sub(" +", " ",s.lower().replace('_',' ').strip())

def parse_request(row):
    return row['question']

def make_df(path,is_test=False):
    """"Предобработка данных"""
    data = pd.read_csv(path)
    df = pd.DataFrame()
    
    df['question'] = data['question']
    df['choices'] = data['choices'].apply(ast.literal_eval)
    
    return df

In [5]:
# Загружаем данные
train_data = make_df('all_datasets.csv')

# Сопоставляем запросы
train_data['request'] = train_data.apply(parse_request,axis=1)

/tmp/ipykernel_1107/1048887473.py:10: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path)


In [6]:
class TextProcessor():
    def __init__(self, kn_base='en_core_web_sm'):
        self.nlp = spacy.load(kn_base)
        self.stop_wrods = stop_words.STOP_WORDS | set(punctuation) | set('123456789')
        
    def prepare_text(self, s):
        """Базовая предобработка текста перед лемматизацией"""
        s = s.lower().replace('_',' ').strip()
        for p in punctuation:
            s = s.replace(p,' ')
        return re.sub(" +", " ",s)
    
    def precess_text(self,s):
        """Лемматизируем запрос и удаляем стоп слова"""
        s = self.prepare_text(s)
        doc = self.nlp(s)
        lemms = [token.lemma_ for token in doc]
        lemms = [token for token in lemms if token not in self.stop_wrods]
        return lemms
    
    def process_batch(self,texts,verbose=True):
        """Для увеличения скорости паралелим вычесления"""
        iterator = tqdm(texts) if verbose else texts
        pool = mp.Pool(mp.cpu_count())
        prep_texts = pool.map(self.precess_text,iterator)
        return prep_texts

In [7]:
class EntityLinlker():
    def __init__(self, kg, text_proc, prepared2concept):
        self.kg = kg
        self.text_proc = text_proc
        self.p2c = prepared2concept
        
    def link_sentence(self,s):
        """Ищем для запроса релевантных кандидатов"""
        candidates = []
        s = self.text_proc.precess_text(s)
        s_st = set(s)
        for i,k in enumerate(self.kg):
            if len(k) == 0:
                continue
            elif len(k) == 1:
                if k[0] in s_st:
                    candidates.append(i)
            else:
                if all(x in s_st for x in k):
                    idxes = [s.index(x) for x in k]
                    if all(idxes[j+1] - idxes[j] == 1 for j in range(len(k)-1)):
                           candidates.append(i)
        return candidates
    
    def batch_linking(self,texts,verbose=True):
        """Для увеличения скорости паралелим вычесления"""
        iterator = tqdm(texts) if verbose else texts
        pool = mp.Pool(mp.cpu_count())
        prep_texts = pool.map(self.link_sentence,iterator)
        return prep_texts

In [9]:
processor = TextProcessor()
graph_state = pickle.load(open('graph.pkl','rb'))
concepts = graph_state['concepts_maper'].keys()
concepts = [(x if type(x) == str else '') for x in concepts] #delete valid nodes

In [10]:
prepared_concepts = pickle.load(open('entity_linker_kg.pkl','rb')) # - если не хотите ждать
concept2prepared = dict(zip(concepts,prepared_concepts)) # словарь, сопоставляющий концепт -> его лемматезированная версия

In [11]:
def texual_acc(s1,s2): # like charackter bleu@1 score
    return sum(i == j for i,j in zip(s1,s2)) / (max(len(s1),len(s2)) + 1e-6)

def prep_text(s):
    return s.lower().strip().replace('_', ' ')

def rank_func(s1,texts):
    s1 = prep_text(s1)
    scores = []
    for s in texts:
        scores.append(texual_acc(s1,prep_text(s)))
    return texts[np.argmax(scores)]

In [12]:
# из за лемматизации, текстовая составляющая некоторых вершин стала идентичной
# по этому для их текста ищем вершину, чья не лемматезированная версия как можно сильнее совподает с не лемматезированной
# таким образом получаем словарь, который сопоставляет лемматизированному концепту его исходную версию
df_cp = pd.DataFrame(zip(concepts,prepared_concepts))
df_cp[1] = df_cp[1].apply(lambda x: ' '.join(x))
df_cp_grp = df_cp.groupby(1)[0].agg(list)
ranked_nodes = [rank_func(y,x) for x,y in tzip(df_cp_grp.tolist(), df_cp_grp.index.tolist())]
prepared2concept = dict(zip(df_cp_grp.index.tolist(),ranked_nodes))

  0%|          | 0/656261 [00:00<?, ?it/s]

In [13]:
linker = EntityLinlker(prepared_concepts, processor, prepared2concept)

In [ ]:
# ищем кандидатов
train_data['candidates'] = linker.batch_linking(train_data['request'].tolist())

  0%|          | 0/509453 [00:00<?, ?it/s]

In [ ]:
train_data['candidates'] 

In [ ]:
def get_concepts(ids):
    """Преобразуем концепты в число"""
    ids = [prepared_concepts[i] for i in ids]
    ids = [' '.join(x) for x in ids]
    return [graph_state['concepts_maper'][prepared2concept[i]] for i in ids] #[concepts.index(prepared2concept[i]) for i in ids] 

In [ ]:
train_data['candidates'] = train_data['candidates'].apply(get_concepts)

In [ ]:
train_data['candidates']

In [ ]:
idf_matrix = pd.Series(
    np.concatenate(train_data['candidates'])
).value_counts()
idf_matrix = idf_matrix.to_dict()

In [33]:
# Берём семплы с наименьшей популярностью, так как они будут более значимыми для нашей модели
# Таким образом получается получить top-k самых редких семплов, о которых исходная LM скорее всего не обладает знаниями
def rank_by_idf(texts,topk=5):
    idx = np.unique(texts, return_index=True)[1]
    texts = [texts[i] for i in sorted(idx)]
    idfs = [idf_matrix[x] for x in texts]
    return [texts[i] for i in np.argsort(idfs)[:topk]]

In [36]:
train_data['candidates_rank'] = train_data['candidates'].apply(rank_by_idf)

# Merge All candidates

In [41]:
bge_state = pickle.load(open('vqa_state_bge','rb'))

In [48]:
bge_state['choices_encodes']

array([17241, 28258,  6586, ..., 22493, 48514, 47639])

In [43]:
concepts = [prepare_text(s)  if type(s) == str else ' ' for s in graph_state['concepts_maper']]
concepts_inv = dict(enumerate(concepts))

In [66]:
question_bge = [[concepts_inv[x] for x in i] for i in bge_state['question_encodes']]
choices_bge = [[concepts_inv[x] for x in bge_state['choices_encodes'][i:i+4]] for i in range(0,len(bge_state['choices_encodes']),4)]

In [71]:
choices_bge[3]

['police', 'firemen', 'airport worker', 'postal workers']

In [75]:
lemma_candides = [[graph_state['concepts_maper_inv'][x] for x in k] for k in train_data['candidates_rank'].tolist()]

In [81]:
def merge_cands(x1,x2,x3):
    all_cands = np.unique(x1 + x2 + x3)
    return all_cands.tolist()

In [82]:
all_cands = [
    merge_cands(x1,x2,x3)
    for x1,x2,x3 in zip(question_bge, lemma_candides, choices_bge)
]

In [84]:
with open('vqa_candidates.pkl','wb') as f:
    pickle.dump(all_cands,f)

In [121]:
```python

class GraphProcessorV3():
    def __init__(self, G, embeds, n_hop=1,pad_to_nodes=64):
        self.G = G
        self.embeds = embeds
        self.pad_to_nodes = pad_to_nodes
        self.n_hop = n_hop
        
    def get_k_hop_neighbours(self,node):
        if self.n_hop == 1:
            return list(self.G.neighbors(node))
        nbrs = set([node])
        for l in range(self.n_hop):
            nbrs |= set((nbr for n in nbrs for nbr in self.G.neighbors(n)))
        return list(nbrs)
    
    def chek_unique(self,nodes):
        """Unique without sort"""
        indexes = np.unique(nodes, return_index=True)[1]
        return [nodes[i] for i in sorted(indexes)]
    
    def process_image(self, nodes, image_triplets, image_nodes):
        if len(np.unique(nodes+image_nodes)) >= self.pad_to_nodes:
            nodes = nodes[:self.pad_to_nodes]
        else:
            for n in nodes:
                nodes += self.get_k_hop_neighbours(n)
                nodes = self.chek_unique(nodes)
                if len(np.unique(nodes+image_nodes)) >= self.pad_to_nodes:
                    break
        if len(nodes) >= self.pad_to_nodes:
            pad_length = self.pad_to_nodes + len(np.unique(nodes+image_nodes)) - len(np.unique(nodes))
            nodes = nodes[:pad_length]
        else:
            pad_length = self.pad_to_nodes + len(np.unique(nodes+image_nodes)) - len(np.unique(nodes))
            nodes += [160892] * (pad_length - len(nodes))
        
        sub_graph = self.G.subgraph(nodes).copy()
        sub_graph.add_nodes_from(image_nodes)
        inv_nodes = {n:i for i,n in enumerate(sub_graph.nodes)}    
        edge_attrs,edge_index = [], []
        for node1,node2,edge_feat in sub_graph.edges(data='relation'):
            edge_index.append([inv_nodes[node1], inv_nodes[node2]])
            edge_attrs.append(edge_feat)
            
        for node1, edge_feat, node2 in image_triplets:
            edge_index.append([inv_nodes[node1], inv_nodes[node2]])
            edge_attrs.append(edge_feat)
        
        nodes = list(sub_graph.nodes)
        nodes = [n if n < 799272 else 160892 for n in nodes]
        return {
            'edge_index': torch.tensor(edge_index).T,
            'edge_attr': torch.tensor(edge_attrs),
            'x': torch.from_numpy(np.stack(self.embeds[nodes])).to(torch.float)
        }

    
    def process(self, nodes):
        if len(nodes) >= self.pad_to_nodes:
            nodes = nodes[:self.pad_to_nodes]
        else:
            for n in nodes:
                nodes += self.get_k_hop_neighbours(n)
                if len(nodes) >= self.pad_to_nodes:
                    break
        if len(nodes) >= self.pad_to_nodes:
            nodes = nodes[:self.pad_to_nodes]
        else:
            nodes += [0] * (self.pad_to_nodes - len(nodes))
            
        inv_nodes = {n:i for i,n in enumerate(nodes)}    
        sub_graph = self.G.subgraph(nodes).copy()
        edge_attrs,edge_index = [], []
        for node1,node2,edge_feat in sub_graph.edges(data='relation'):
            edge_index.append([inv_nodes[node1], inv_nodes[node2]])
            edge_attrs.append(edge_feat)
        nodes = [n if n < 799272 else 160892 for n in nodes]
        return {
            'edge_index': torch.tensor(edge_index).T,
            'edge_attr': torch.tensor(edge_attrs),
            'x': torch.from_numpy(np.stack(self.embeds[nodes])).to(torch.float)
        }

```

In [122]:
graph_state['embeds']

array([[ 0.14546944,  0.06144163,  0.58076036, ..., -0.23021483,
        -0.06329279,  0.12431484],
       [ 0.03331704,  0.10365178,  0.7706704 , ..., -0.05123961,
        -0.28970563, -0.06316865],
       [-0.279177  ,  0.1306568 ,  0.15783656, ..., -0.273939  ,
        -0.32336122,  0.12012992],
       ...,
       [-0.22461623, -0.4128434 , -0.4113543 , ..., -0.3143878 ,
         0.15477931,  0.6069955 ],
       [ 0.42082188,  0.07282558,  0.01273101, ...,  0.12235828,
        -0.22267364,  0.28397152],
       [ 0.10134428,  0.40544418,  0.13394867, ...,  0.14023907,
         0.36362907,  0.4318809 ]], dtype=float32)

In [127]:
import torch

In [123]:
proc = GraphProcessorV3(G=graph_state['graph'],embeds=graph_state['embeds'])

In [129]:
out = proc.process_image([30,52,239,777,1488],[[0,[0]*17,1]],[0,1])

In [133]:
out['edge_index'].T

tensor([[ 1, 58],
        [ 1, 60],
        [ 1, 56],
        [ 1, 62],
        [ 2, 56],
        [ 2,  1],
        [ 3, 12],
        [ 6, 30],
        [ 6, 25],
        [12,  3],
        [18, 12],
        [18, 34],
        [19, 43],
        [19, 44],
        [19, 53],
        [19, 20],
        [19, 21],
        [20, 19],
        [21, 53],
        [23, 30],
        [23, 49],
        [24, 30],
        [24, 29],
        [24, 49],
        [25, 49],
        [29, 30],
        [29, 49],
        [29, 24],
        [30, 29],
        [30, 45],
        [30, 46],
        [30, 31],
        [30, 23],
        [30, 24],
        [30, 25],
        [30, 35],
        [30, 17],
        [30, 28],
        [30, 12],
        [30, 55],
        [30, 34],
        [30, 22],
        [30,  6],
        [30, 32],
        [31, 30],
        [31, 12],
        [31, 34],
        [32, 30],
        [32, 25],
        [32, 12],
        [32, 34],
        [33, 50],
        [34, 49],
        [34, 12],
        [36, 56],
        [3

In [125]:
import networkx as nx
graph = nx.DiGraph()
graph.add_nodes([0,1,2,3])

AttributeError: 'DiGraph' object has no attribute 'add_nodes'

In [88]:
graph.add_nodes_from([0,1,2,3,4,5,6,7,8])

In [93]:
graph.add_nodes_from([7,8,9,10,'hui',])

In [94]:
nx.

NodeView((0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 'hui'))